In [1]:
import sys
sys.path.append('C:/Work/pythonshizai/07_src')

In [2]:
import numpy as np
import pandas as pd
import argparse
import inject
import sys
import os
from configparser import ConfigParser

In [3]:
import tensorflow as tf

In [4]:
from C7013.lib import const, message
from C7013.lib.C7013_04_rank_training_task import C7013_04_rank_training_task
from C7013.lib.rank_model_helper import RankModelHelper
from C7013.lib.rank_model_helper import RankTrainingGenerator
from C7013.lib.C7013_04_rank_flag_forced_correction_task import C7013_04_rank_flag_forced_correction_task 
from C7013.lib import rank_utils
from C7013.lib import onehot_utils

In [5]:
print('ランク検証タスクを実行します。')

ランク検証タスクを実行します。


In [6]:
rank_model_helper: RankModelHelper = RankModelHelper()

In [7]:
batch_size = const.APP_CONFIG['rank_config']['batch_size']

In [8]:
SHUFFLE_BUFFER_SIZE = 100

In [10]:
rank_a_model = tf.keras.models.load_model('C:/Work/PythonShizai/07_src/C7013/data/rank_model/a/', compile=True)
rank_b_model = tf.keras.models.load_model('C:/Work/PythonShizai/07_src/C7013/data/rank_model/b/', compile=True)
rank_c_model = tf.keras.models.load_model('C:/Work/PythonShizai/07_src/C7013/data/rank_model/c/', compile=True)
rank_d_model = tf.keras.models.load_model('C:/Work/PythonShizai/07_src/C7013/data/rank_model/d/', compile=True)
rank_bar_model = tf.keras.models.load_model('C:/Work/PythonShizai/07_src/C7013/data/rank_model/bar/', compile=True)

In [11]:
rank_a_model.summary()

Model: "rank_a_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rank_input (InputLayer)      [(None, 83)]              0         
_________________________________________________________________
middle_layer1 (Dense)        (None, 55)                4620      
_________________________________________________________________
middle_layer2 (Dropout)      (None, 55)                0         
_________________________________________________________________
rank_output (Dense)          (None, 1)                 56        
Total params: 4,676
Trainable params: 4,676
Non-trainable params: 0
_________________________________________________________________


In [12]:
test_file_path = 'C:/Work/PythonShizai/07_src/C7013/data/rank_verification.csv'

In [13]:
test_data = pd.read_csv(test_file_path, sep=',', encoding='utf-8')
if test_data.empty:
    raise RuntimeError(f"データが存在しません。file_path={test_file_path}")
flag_forced_correction = C7013_04_rank_flag_forced_correction_task()
_test_data: pd.DataFrame = flag_forced_correction.execute(test_data)

[2021-03-03 17:39:59,152][INFO    ](C7013_04_rank_flag_forced_correction_task.py:37) フラグ強制補正タスクを実行します。


In [14]:
test_examples = rank_utils.input_data_transform(_test_data)

In [15]:
target_onehot = rank_utils.target_data_transform(_test_data)

In [16]:
test_labels_a = target_onehot[:,0:1]
test_labels_b = target_onehot[:,1:2]
test_labels_c = target_onehot[:,2:3]
test_labels_d = target_onehot[:,3:4]
test_labels_bar = target_onehot[:,4:5]

In [17]:
test_labels_a.shape

(21516, 1)

In [18]:
test_dataset_a = tf.data.Dataset.from_tensor_slices((test_examples, test_labels_a))
test_dataset_b = tf.data.Dataset.from_tensor_slices((test_examples, test_labels_b))
test_dataset_c = tf.data.Dataset.from_tensor_slices((test_examples, test_labels_c))
test_dataset_d = tf.data.Dataset.from_tensor_slices((test_examples, test_labels_d))
test_dataset_bar = tf.data.Dataset.from_tensor_slices((test_examples, test_labels_bar))

In [19]:
test_dataset_a = test_dataset_a.batch(batch_size)
test_dataset_b = test_dataset_b.batch(batch_size)
test_dataset_c = test_dataset_c.batch(batch_size)
test_dataset_d = test_dataset_d.batch(batch_size)
test_dataset_bar = test_dataset_bar.batch(batch_size)

In [27]:
rank_a_model.evaluate(test_dataset_a)

673/673 [==============================] - 1s 2ms/step - loss: 1.1025e-04 - accuracy: 0.9999


[0.00011025253479601815, 0.9999070167541504]

In [28]:
rank_b_model.evaluate(test_dataset_b)

673/673 [==============================] - 1s 2ms/step - loss: 0.0109 - accuracy: 0.9962


[0.01089794933795929, 0.9962353706359863]

In [29]:
rank_c_model.evaluate(test_dataset_c)

673/673 [==============================] - 1s 1ms/step - loss: 0.0534 - accuracy: 0.9792


[0.0533558651804924, 0.9791783094406128]

In [30]:
rank_d_model.evaluate(test_dataset_d)

673/673 [==============================] - 1s 2ms/step - loss: 0.0825 - accuracy: 0.9687


[0.08246687799692154, 0.9687209725379944]

In [31]:
rank_bar_model.evaluate(test_dataset_bar)

673/673 [==============================] - 1s 1ms/step - loss: 0.0549 - accuracy: 0.9779


[0.054949283599853516, 0.9779233932495117]

In [20]:
rank_a_predict = np.round(rank_a_model.predict(test_dataset_a, verbose=1)).astype(np.int32)

673/673 [==============================] - 1s 1ms/step


In [21]:
rank_b_predict = np.round(rank_b_model.predict(test_dataset_b, verbose=1)).astype(np.int32)

673/673 [==============================] - 1s 1ms/step


In [22]:
rank_c_predict = np.round(rank_c_model.predict(test_dataset_c, verbose=1)).astype(np.int32)

673/673 [==============================] - 1s 1ms/step


In [23]:
rank_d_predict = np.round(rank_d_model.predict(test_dataset_d, verbose=1)).astype(np.int32)

673/673 [==============================] - 1s 1ms/step


In [24]:
rank_bar_predict = np.round(rank_bar_model.predict(test_dataset_bar, verbose=1)).astype(np.int32)

673/673 [==============================] - 1s 1ms/step


In [25]:
result = rank_utils.decide_rank(rank_a_predict, rank_b_predict, rank_c_predict, rank_d_predict, rank_bar_predict)

In [26]:
result

array([500, 500,   0, ..., 500, 500, 500])